<a href="https://colab.research.google.com/github/CISC452MLProject/CISC-452-Group-22/blob/main/group22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Custom Trained Model

In [1]:
import os
from mmdet.apis import inference_detector, init_detector
import requests
from mmpose.utils import adapt_mmdet_pipeline
from mmpose.registry import VISUALIZERS
from mmpose.apis import inference_topdown
from mmpose.apis import init_model as init_pose_estimator
import mmcv
import numpy as np
from mmpose.evaluation.functional import nms
from mmpose.structures import merge_data_samples

In [2]:
dec_checkpoint_url = "https://download.openmmlab.com/mmpose/v1/projects/rtmpose/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth"
pos_config_url = "https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-body7_pt-body7_420e-256x192-e48f03d0_20230504.pth"

detector = init_detector(
        "mmpose/demo/mmdetection_cfg/rtmdet_m_640-8xb32_coco-person.py", dec_checkpoint_url, device="cuda")
detector.cfg = adapt_mmdet_pipeline(detector.cfg)


pose_estimator = init_pose_estimator(
        "mmpose/configs/body_2d_keypoint/rtmpose/body8/rtmpose-m_8xb256-420e_body8-256x192.py ",
        pos_config_url,
        device="cuda",
        cfg_options=dict(
            model=dict(test_cfg=dict(output_heatmaps=True))))

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmpose/rtmdet_m_8xb32-100e_coco-obj365-person-235e8209.pth
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmpose/v1/projects/rtmposev1/rtmpose-m_simcc-body7_pt-body7_420e-256x192-e48f03d0_20230504.pth


In [3]:
pose_estimator.cfg.visualizer.radius = 3
pose_estimator.cfg.visualizer.alpha = 0.8
pose_estimator.cfg.visualizer.line_width = 1
visualizer = VISUALIZERS.build(pose_estimator.cfg.visualizer)

visualizer.set_dataset_meta(
    pose_estimator.dataset_meta, skeleton_style="mmpose")

In [ ]:
img = "img\\49_2020-11-28_2-18.jpg"
det_result = inference_detector(detector, img)
pred_instance = det_result.pred_instances.cpu().numpy()
bboxes = np.concatenate(
    (pred_instance.bboxes, pred_instance.scores[:, None]), axis=1)
bboxes = bboxes[np.logical_and(pred_instance.labels == 0,
                                pred_instance.scores > 0.3)]
bboxes = bboxes[nms(bboxes, 0.3), :4]

In [27]:
pose_results = inference_topdown(pose_estimator, img, bboxes)
data_samples = merge_data_samples(pose_results)

print(pose_results)
print(data_samples)

[<PoseDataSample(

    META INFORMATION
    input_center: array([613.4508 , 625.46246], dtype=float32)
    pad_shape: (256, 192)
    input_scale: array([441.4745, 588.6326], dtype=float32)
    flip_indices: [0, 2, 1, 4, 3, 6, 5, 8, 7, 10, 9, 12, 11, 14, 13, 16, 15]
    input_size: (192, 256)
    img_path: None
    ori_shape: (1080, 1920)
    batch_input_shape: (256, 192)
    img_shape: (1080, 1920)
    dataset_name: 'coco'

    DATA FIELDS
    pred_instances: <InstanceData(
        
            META INFORMATION
        
            DATA FIELDS
            keypoint_y_labels: array([[[0.03134083, 0.00073897, 0.        , ..., 0.        ,
                         0.        , 0.        ],
                        [0.24024434, 0.18790248, 0.14666435, ..., 0.00581487,
                         0.02994253, 0.06080639],
                        [0.2798652 , 0.22456323, 0.18130606, ..., 0.02159458,
                         0.04341634, 0.07043859],
                        ...,
                      

In [28]:
img = mmcv.imread(img, channel_order='rgb')
visualizer.add_datasample(
            'result',
            img,
            data_sample=data_samples,
            draw_gt=False,
            draw_heatmap=True,
            draw_bbox=True,
            skeleton_style="mmpose",
            show=True)

array([[[  8,   9,  14],
        [  8,   9,  14],
        [  8,   9,  14],
        ...,
        [  6,   8,  21],
        [  6,   8,  21],
        [  6,   8,  21]],

       [[  7,   8,  13],
        [  7,   8,  13],
        [  7,   8,  13],
        ...,
        [  6,   8,  21],
        [  6,   8,  21],
        [  6,   8,  21]],

       [[  7,   8,  13],
        [  7,   8,  13],
        [  7,   8,  13],
        ...,
        [  6,   8,  21],
        [  6,   8,  21],
        [  6,   8,  21]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]